In [1]:
import argparse
from ast import parse
import os
import time
from retrievalHelper.utils import *
from kwExtractorHelper.utils import mkdir
from retrievalHelper.u4Res import *
from retrievalHelper.u4KNN import *
from retrievalHelper.u4train import *
from retrievalHelper.models import *

D:\anaconda\envs\dangkh\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
random_seed = 1001
setSeed(random_seed)

listcity = ['charlotte', 'edinburgh', 'lasvegas', 'london', 'phoenix', 'pittsburgh', 'singapore']
city = "tripAdvisor"

In [3]:
trainDat, testDat = data_reviewLoader(city)
train_users, train_users2kw = extract_users(trainDat['np2users'])
test_users, test_users2kw = extract_users(testDat['np2users'])

info from extracted file
Number of keyword: 49928
info from extracted file
Number of keyword: 125955


In [4]:
edgeType = "IUF"
quantity = 20

In [5]:
gt = load_groundTruth(f'./data/reviews/{city}.csv')

number of review:  84904


In [6]:
keywordScore, keywordFrequence = load_kwScore(city, edgeType)
restGraph = retaurantReviewG([trainDat, keywordScore, keywordFrequence, \
                                quantity,  edgeType, gt])

KNN = neighbor4kw(f'{city}_kwSenEB_pad', testDat,  restGraph)
rest_Label = getRestLB(trainDat['np2rests'])

prediction = []

# JACCARD

In [ ]:
print("*"*50)
print("using Jaccard")
print("*"*50)
jcsim = JaccardSim(f'./data/score/{city}-keywords-TFIUF.json', quantity)
lResults = []
for idx in tqdm(range(len(test_users))):
    testUser = test_users[idx]
    testkey = test_users2kw[idx]
    pred = jcsim.pred(testkey)
    groundtruth = gt[testUser]
    prediction.append(pred)
    score = quick_eval(pred, groundtruth, sourceFile, city=='tripAdvisor')
    lResults.append(score)
mp, mr, mf = extractResult(lResults)

# MF

In [34]:
print("*"*50)
print("running MF-BPR at: ")
print("*"*50)

numUser = len(train_users)
numItem = len(rest_Label)

label_train, label_test = label_ftColab(train_users, test_users, gt, restGraph.numRest, rest_Label)

learning_rate = 0.03
embedding_dim = 32
num_epochs = 30



**************************************************
running MF-BPR at: 
**************************************************


100%|████████████████████████████████████████████████████████████████████████████| 2394/2394 [00:00<00:00, 9564.51it/s]


In [35]:
trainLB = np.asarray(label_train)
testLB = np.asarray(label_test)
train_dataset = DataBPR(trainLB)
test_dataset = DataBPR(trainLB, True)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [36]:
model = MFBPR(numUser, numItem, embedding_dim).to(device)

mp, mr, mf = 0, 0, 0
print(model)
optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay= 1e-4)
# Training loop
for epoch in range(num_epochs):
    total_loss = 0.0
    model.train()
    for batch_idx, batchDat in enumerate(train_loader):
        uid, pid, nid = batchDat
        uid = uid.to(device)
        pid = pid.to(device)
        nid = nid.to(device)

        optimizer.zero_grad()
        pred_i, pred_j = model(uid, pid, nid)
        loss = - ((pred_i - pred_j).sigmoid()+1e-10).log().sum()
        loss.backward()
        total_loss += loss.item() 
        optimizer.step()

        # Print training progress
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss}')

    if (epoch+1) % 10 == 0:
        
        lResults = evaluateModel_MFBPR(model, test_loader, "", gt, train_users, quantity, rest_Label)
        p, r, f = extractResult(lResults)
        if mean(r) > mean(mr):
            mp, mr, mf = p, r, f
        print(f'Epoch [{epoch+1}/{num_epochs}], prec: {mean(p)}, rec: {mean(r)}, f1: {mean(f)}')



MFBPR(
  (embed_user): Embedding(9499, 32)
  (embed_item): Embedding(3097, 32)
)
Epoch [1/30], Loss: 58735.69380950928
Epoch [2/30], Loss: 13745.461366653442
Epoch [3/30], Loss: 1408.0112488865852
Epoch [4/30], Loss: 138.45384038984776
Epoch [5/30], Loss: 39.52012599632144
Epoch [6/30], Loss: 25.013183349743485
Epoch [7/30], Loss: 18.873755887150764
Epoch [8/30], Loss: 14.65679629612714
Epoch [9/30], Loss: 11.49552525114268
Epoch [10/30], Loss: 9.065094801597297
Epoch [10/30], prec: 0.06402865711429524, rec: 0.1812733477164979, f1: 0.09131161103207273
Epoch [11/30], Loss: 7.164304377511144
Epoch [12/30], Loss: 5.665467691142112
Epoch [13/30], Loss: 4.479306990280747
Epoch [14/30], Loss: 3.535994851961732
Epoch [15/30], Loss: 2.78547325078398
Epoch [16/30], Loss: 2.190803668112494
Epoch [17/30], Loss: 1.7198301464086398
Epoch [18/30], Loss: 1.3474361373810098
Epoch [19/30], Loss: 1.0533685258124024
Epoch [20/30], Loss: 0.822283276123926
Epoch [20/30], prec: 0.06976007997334223, rec: 0.1

  0%|                                                                                         | 0/2394 [00:00<?, ?it/s]


ValueError: too many values to unpack (expected 3)

In [ ]:
lResults = []
for idx in tqdm(range(len(test_users))):
    if idx > 3000:
        break
    testUser, topK_Key, _, topUser = procesTest(test_users, test_users2kw, idx, KNN, restGraph, True)
    idU = torch.LongTensor([train_users.index(x) for x in topUser]).to(device)
    rest_score = []
    for rest in range(restGraph.numRest):
        restID = torch.LongTensor([rest]).to(device)
        rest_score.append(model.csPrediction(idU,restID))
    
    tmp = np.argsort(rest_score)[::-1][:20]
    restPred = [rest_Label[x] for x in tmp]

    groundtruth = gt[testUser]
    score = quick_eval(restPred, groundtruth, None, city=='tripAdvisor')
    lResults.append(score)
mp, mr, mf = extractResult(lResults)

  2%|█▍                                                                            | 46/2394 [02:04<1:46:15,  2.72s/it]

In [ ]:
p, r, f = mp, mr, mf 
print(mean(p), mean(r), mean(f))

# MPG

In [11]:
print("*"*50)
print("using MPG")
print("*"*50)
l_rest = restGraph.listRestCode
kw_data = KNN.kw_data
lu, li, lh = list([]), list([]), list([])
for rest in tqdm(keywordScore):
    kw_scs = keywordScore[rest]
    u = l_rest.index(rest)
    tmpI, tmpH = [], []
    for kw, sc in kw_scs:
        tmpI.append(kw_data.index(kw))
        tmpH.append(sc)
    tmpU = [u] * len(tmpI)
    lu.extend(tmpU)
    li.extend(tmpI)
    lh.extend(tmpH)

**************************************************
using MPG
**************************************************


100%|██████████████████████████████████████████████████████████████████████████████| 3097/3097 [01:27<00:00, 35.20it/s]


In [13]:
adj = np.zeros([len(l_rest), len(kw_data)])
for ii in range(len(lu)):
    u, v, w = lu[ii], li[ii], lh[ii]
    adj[u, v] = w

lResults = []
lidx = [x for x in range(len(test_users))]
np.random.shuffle(lidx)
dictionary = {}
listsimU = []

In [23]:
def quick_eval2(preds, gt, source = None):
    '''
    - preds: [list of restaurants]
    - GT: [('wrdLrTcHXlL4UsiYn3cgKQ', 4.0), ('uG59lRC-9fwt64TCUHnuKA', 3.0)]
    - 
    '''
    gt_list = set([str(a[0]) for a in gt])
    # print(gt_list)
    preds_list = list(set(preds))
    ov = gt_list.intersection(preds_list)
    # print("*"*20)
    # print(ov)
    # print("*"*20)
    prec = len(ov)/len(preds_list)
    rec = len(ov)/len(gt_list)
    f1 = 0 if prec+rec == 0 else 2*prec*rec/(prec+rec)
    # if source != None :
    #     print("Precision: {}, Recall: {}, F1: {}".format(prec, rec, f1), file = source)
    return prec, rec, f1

In [84]:
for ite in tqdm(range(len(test_users))):
    idx = lidx[ite]
    testUser, topK_Key, keyfrequency, topUser = procesTest(test_users, test_users2kw, idx, KNN, restGraph, False)
    testkey = [kw_data.index(x) for x in topK_Key]
    ft = np.zeros(len(kw_data))
    for x in testkey: ft[x] = 1.0
    ft = ft.reshape(-1, 1)
    tmp = np.matmul(adj, ft).reshape(-1)
    idxrest = np.argsort(tmp)[::-1]
    result = [l_rest[x] for x in idxrest[:quantity]]
    prediction.append(result)
    groundtruth = gt[testUser]
    print(result, groundtruth)
    score = quick_eval2(result, groundtruth)
    print(score)
    stop
    lResults.append(score)

  0%|                                                                                         | 0/2394 [00:00<?, ?it/s]


['223023', '225108', '87617', '641273', '87638', '223061', '114581', '93618', '220240', '259593', '208455', '84115', '87620', '114595', '87657', '87590', '91042', '87645', '87592', '217498'] [(208455, 5.0), (87656, 3.0), (223061, 4.0), (87659, 4.0), (99535, 5.0), (259593, 5.0)]
(0.15, 0.5, 0.23076923076923075)


NameError: name 'stop' is not defined

In [81]:
mp, mr, mf = extractResult(lResults)
mean(mp), mean(mr), mean(mf)

(0.13180868838763576, 0.3948970000249251, 0.19182999295526143)

In [92]:
for ite in tqdm(range(len(test_users))):
    idx = lidx[ite]
    testUser, topK_Key, keyfrequency, topUser = procesTest(test_users, test_users2kw, idx, KNN, restGraph, False)
    testkey = [kw_data.index(x) for x in topK_Key]
    ft = np.zeros(len(kw_data))
    for x in testkey: ft[x] = 1.0
    ft = ft.reshape(-1, 1)
    tmp = np.matmul(adj, ft).reshape(-1)
    sc = tmpHelper.query(testUser)
    tmp = tmp*sc
    idxrest = np.argsort(tmp)[::-1]
    result = [l_rest[x] for x in idxrest[:quantity]]
    prediction.append(result)
    groundtruth = gt[testUser]
    print(result, groundtruth)
    score = quick_eval2(result, groundtruth)
    print(score)
    stop
    lResults.append(score)

  0%|                                                                                         | 0/2394 [00:00<?, ?it/s]

[60763, 35805, 60956, 55711]
['223023', '225108', '87617', '87638', '223061', '114581', '93618', '220240', '259593', '208455', '87620', '114595', '87657', '87590', '91042', '87645', '87592', '217498', '111487', '114577'] [(208455, 5.0), (87656, 3.0), (223061, 4.0), (87659, 4.0), (99535, 5.0), (259593, 5.0)]
(0.15, 0.5, 0.23076923076923075)


NameError: name 'stop' is not defined

In [83]:
mp, mr, mf = extractResult(lResults)
mean(mp), mean(mr), mean(mf)

(0.13635477582846003, 0.40906758577225427, 0.19852822307281182)

In [90]:
class regionHelper(object):
    """docstring for regionHelper"""
    def __init__(self):
        super(regionHelper, self).__init__()
        df2 = pd.read_csv('./data/reviews/hotel.csv')
        hotel_region_dict = dict(zip(df2['hotelID'], df2['region_id']))
        rest2city = []
        for rest in l_rest:
            rest = int(rest)
            rest2city.append(hotel_region_dict[rest])
        self.rest2city = np.asarray(rest2city)
        with open('./data/reviews/user2region.json', 'r') as f:
            self.author_region = json.load(f)


    def query(self, idx):
        tmp_reg = self.author_region[str(idx)]
        print(tmp_reg)
        marker = np.zeros(len(self.rest2city))
        for reg in tmp_reg:
            tmp = np.where(self.rest2city == reg)[0]
            marker[tmp] = 1
        return marker
        

In [91]:
tmpHelper = regionHelper()

# CBR

In [30]:
print("*"*50)
print("Using CBR")
print("*"*50)
dim_users, dim_items = 384, 384
learning_rate = 0.03
hidden_dim = 32
num_epochs = 500

**************************************************
Using CBR
**************************************************


In [29]:

userFT, userFT_test, rest_feature = KNN.loadFT(20, rest_Label, city)
label_train, label_test = label_ftColab(train_users, test_users, gt, restGraph.numRest, rest_Label)



trainLB = np.asarray(label_train)
testLB = np.asarray(label_test)


**************************************************
Using CBR
**************************************************


100%|███████████████████████████████████████████████████████████████████████████| 9499/9499 [00:00<00:00, 16142.30it/s]


KeyboardInterrupt: 

In [24]:
nbatch_size = 128

print("feature shape (train / test):")
print(userFT.shape, userFT_test.shape)
train_dataset = DataCF(userFT, trainLB)
test_dataset = DataCF(userFT_test, testLB)


train_loader = DataLoader(train_dataset, batch_size=nbatch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=nbatch_size, shuffle=False)
rest_feature = torch.from_numpy(rest_feature).type(torch.FloatTensor)
rest_feature = rest_feature.to(device)



feature shape (train / test):
(9499, 20, 384) (2394, 20, 384)


TypeError: expected np.ndarray (got Tensor)

In [31]:
model = CBR(dim_users, dim_items, hidden_dim).to(device)

In [32]:
# Model, optimizer, and loss function

mp, mr, mf = 0, 0, 0
print(model)
# stop
optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay= 1e-4)
criterion = nn.BCELoss()
# Training loop
for epoch in range(num_epochs):
    total_loss = 0.0
    model.train()
    for batch_idx, batchDat in enumerate(train_loader):
        optimizer.zero_grad()
        data, label = batchDat
        userDat = data.to(device)
        label = label.to(device)
        restDat = rest_feature
        prediction = model(userDat, restDat)
        loss = criterion(prediction , label)

        # Backward pass and optimization step
        loss.backward()
        total_loss += loss.item() 
        optimizer.step()

        # Print training progress
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss}')

    if (epoch+1) % 10 == 0:
        lResults = evaluateModel(model, test_loader, rest_feature, gt, test_users, 20, rest_Label)
        p, r, f = extractResult(lResults)
        if mean(r) > mean(mr):
            mp, mr, mf = p, r, f
        print(f'Epoch [{epoch+1}/{num_epochs}], prec: {mean(p)}, rec: {mean(r)}, f1: {mean(f)}')

CBR(
  (user_embeddings): Linear(in_features=384, out_features=32, bias=True)
  (uAP): AttentionPooling(
    (V): Linear(in_features=32, out_features=8, bias=True)
    (w): Linear(in_features=8, out_features=1, bias=True)
    (tanh): Sigmoid()
    (softmax): Softmax(dim=1)
  )
  (item_embeddings): Linear(in_features=384, out_features=32, bias=True)
  (iAP): AttentionPooling(
    (V): Linear(in_features=32, out_features=8, bias=True)
    (w): Linear(in_features=8, out_features=1, bias=True)
    (tanh): Sigmoid()
    (softmax): Softmax(dim=1)
  )
  (dropout): Dropout(p=0.2, inplace=False)
  (activate): Sigmoid()
)
Epoch [1/500], Loss: 3.1844739066436887
Epoch [2/500], Loss: 1.1400064500048757
Epoch [3/500], Loss: 1.1156885456293821
Epoch [4/500], Loss: 1.0991381723433733
Epoch [5/500], Loss: 1.0889850594103336
Epoch [6/500], Loss: 1.084524480625987
Epoch [7/500], Loss: 1.0812150817364454
Epoch [8/500], Loss: 1.0785928899422288
Epoch [9/500], Loss: 1.0754112675786018
Epoch [10/500], Loss:

KeyboardInterrupt: 

In [33]:
p, r, f = mp, mr, mf 
print(mean(p), mean(r), mean(f))

0.07222222222222223 0.21671043334975734 0.10516137948747117
